# **Pré-requisitos**

In [1]:
# Instalar dependências do python
%pip install nltk
%pip install spacy
!python -m spacy download pt_core_news_sm
#!python -m spacy download pt_core_news_lg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 28.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
# Importar bibliotecas necessárias para este projeto
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import spacy

In [3]:
# Instalar dependências do nltk e spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Permitir visualizar toda a ementa das leis
pd.set_option('display.max_colwidth', None)

# **Carregando os dados**
Para este trabalho, vamos utilizar somente as ementas das leis.

In [29]:
# Carregar a base de dados mais atual
df = pd.read_csv('data/Leis de Sorocaba (Clean).csv')
df.fillna('', inplace=True)
df.head()

,summary
0,dispõe sobre denominação de josé antonio pascoto a uma via pública
1,dispõe sobre denominação de vitor hage a uma via pública e dá outras providências
2,dispõe sobre denominação de lisardo cunha dias a uma via pública e dá outras providências
3,dispõe sobre denominação de desirre ferraz cardoso a uma via pública de nossa cidade e dá outras providências
4,institui no calendário oficial do município o dia do sociólogo e dá outras providências


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12812 entries, 0 to 12811
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summary  12812 non-null  object
dtypes: object(1)
memory usage: 100.2+ KB


# **Processamento baseado no vocabulário**

## **Remoção de stopwords**

In [7]:
# Recuperar lista de stopwords em português
stopwords = nltk.corpus.stopwords.words('portuguese')

# Remover stopwords
def remove_stopwords(text):
  words = word_tokenize(text)

  stop = set(stopwords)
  filtered_sentence = [w for w in words if w not in stop]

  # Juntar os tokens novamente em formato de texto
  return " ".join(filtered_sentence)

In [8]:
remove_stopwords(df['summary'][0])

'dispõe sobre denominação josé antonio pascoto via pública'

In [9]:
remove_stopwords(df['summary'][4])

'institui calendário oficial município dia sociólogo dá outras providências'

In [10]:
remove_stopwords(df['summary'][23])

'institui âmbito município sorocaba programa melhorias habitacionais reforma moradias inseridas zonas áreas especial interesse social aeis/zeis dá outras providências'

In [11]:
remove_stopwords(df['summary'][214])

'declara utilidade pública associação classic sports a.c.s dá outras providências'

In [12]:
remove_stopwords(df['summary'][2000])

'dispõe sobre revogação lei Nº # TOKEN_DATA dá outras providências'

In [13]:
remove_stopwords(df['summary'][10699])

'dá nova redação artigo # lei Nº # TOKEN_DATA'

## **Lemmatização**

In [14]:
# Remover stopwords
def lemmatize(text):
  nlp = spacy.load("pt_core_news_sm")
  doc = nlp(text)
  lemmatized_sentence = [token.lemma_ for token in doc]

  # Juntar os tokens novamente em formato de texto
  return " ".join(lemmatized_sentence)

In [15]:
lemmatize(df['summary'][0])

'dispor sobre denominação de José Antonio pascoto a um via público'

In [16]:
lemmatize(df['summary'][4])

'instituir em o calendário oficial de o município o dia de o sociólogo e dar outro providência'

In [17]:
lemmatize(df['summary'][23])

'instituir em o âmbito de o município de sorocaba o programa de melhoria habitacional para reforma de moradia inserir em zona ou área de especial interesse social ael / zel e dar outro providência'

In [18]:
lemmatize(df['summary'][214])

'declarar de utilidade público o Associação Classic Sports a.c.s e dar outro providência'

In [19]:
lemmatize(df['summary'][2000])

'dispor sobre o revogação de o lei Nº   #   de TOKEN_DATA e dar outro providência'

In [20]:
lemmatize(df['summary'][10699])

'dar novo redação a o artigo   #   de o lei Nº   #   de TOKEN_DATA'

## **Aplicando no dataset**

In [21]:
nlp = spacy.load("pt_core_news_sm")
stop = set([word for word in nltk.corpus.stopwords.words('portuguese')])

def lemmatize_and_remove_stopwords(text):
    # Processa o texto com o Spacy
    doc = nlp(text)
    # Removendo stopwords e depois lematizando
    sentence = [token.lemma_ for token in doc if token.text not in stop]
    # Juntar os tokens novamente em formato de texto
    return pd.Series(" ".join(sentence))


In [22]:
df['summary'][2000]

'dispõe sobre a revogação da lei Nº  #  de TOKEN_DATA e dá outras providências'

In [23]:
lemmatize_and_remove_stopwords(df['summary'][2000])

0    dispor sobre revogação lei Nº   #   TOKEN_DATA dar outro providência
dtype: object

In [31]:
df['summary'] = df['summary'].apply(remove_stopwords)

In [32]:
df

,summary
0,dispõe sobre denominação josé antonio pascoto via pública
1,dispõe sobre denominação vitor hage via pública dá outras providências
2,dispõe sobre denominação lisardo cunha dias via pública dá outras providências
3,dispõe sobre denominação desirre ferraz cardoso via pública cidade dá outras providências
4,institui calendário oficial município dia sociólogo dá outras providências
...,...
12807,dispõe sobre abertura crédito especial $ TOKEN_VALOR
12808,dispõe sobre concessão auxílio dá outras providências
12809,dispõe sobre concessão licença prêmio funcionários municipais
12810,dispõe sobre apreensão eliminação animais


# **Salvando o resultado final**

In [33]:
# Salvar o dataset
df.to_csv('data/Leis de Sorocaba (Preprocessed).csv', index=False)